# Assignment 7: Predictive Modeling of Housing Prices in Philadelphia

**NOTE: THIS ASSIGNMENT IS REQUIRED**.

**Due date: Tuesday, 11/26 by the end of class**

The notebook should be submitted to your own private repository on GitHub, which can be created using the following link:

https://classroom.github.com/a/UczbUUOS

## Part 1: Proposal for the final project

Submit a short (1/2 to 1 page) proposal that outlines and describe your plan for the final project. The proposal should discuss the following aspects of the project:

- the data set(s) you wish to use in the project,
- the questions that you want to explore
- the analysis methods and techniques that you will use 
- how the above items satisfy the requirements outlined in the [final project description](https://github.com/MUSA-620-Fall-2019/final-project)

The description of the final project can be found [here](https://github.com/MUSA-620-Fall-2019/final-project).

**Note:** if you are struggling to come up with ideas for datasets, analyses, etc, please reach out and we can discuss potential project ideas.


#### Deliverable

Upload a word document, PDF, etc to the assignment's GitHub repository with your proposal.

#### Note
For the moment, do not worry too much about how the visualizations/interactive maps will be hosted on the web. We will cover several methods for embedding your visualizations on the web in the coming weeks. These methods fall generally into three categories:

- embedding interactive altair/holoviews plots produced in Python
- creating an interactive dashboard of plots using a free-tier server to host your code

Much more on these methods in the coming weeks!


## Part 2: Modeling Philadelphia's Housing Prices and Algorithmic Fairness


### 2.1 Load data from the Office of Property Assessment

Use `carto2gpd` to load data for properties in Philadelphia that had their last sale during 2018.

Sources: 
- [OpenDataPhilly](https://www.opendataphilly.org/dataset/opa-property-assessments)
- [Metadata](http://metadata.phila.gov/#home/datasetdetails/5543865f20583086178c4ee5/representationdetails/55d624fdad35c7e854cb21a4/)

### 2.2 Load data for census tracts and neighborhoods

Load various Philadelphia-based regions we will use in our analysis.

- Census tracts can be downloaded from: http://data.phl.opendata.arcgis.com/datasets/8bc0786524a4486bb3cf0f9862ad0fbf_0.geojson
- Neighborhoods can be downloaded from the following ESRI Map Server:
https://services.arcgis.com/fLeGjb7u4uXqeF9q/ArcGIS/rest/services/Philly_Neighborhoods/FeatureServer/0

**Note:** For the neighborhoods, you will need to use the `esri2gpd` package to query the Map Server API

### 2.3 Spatially join the sales data and neighborhoods/census tracts.

Perform a spatial join, such that each sale has an associated neighborhood and census tract.

**Note:** after performing the first spatial join, you will need to use the `drop()` function to remove the `index_right` column; otherwise an error will be raised on the second spatial join about duplicate columns.

### 2.4 Train a Random Forest on the sales data

You should follow the steps outlined in lecture to preprocess and train your model. 

**Extra credit: the students with the top 3 scores on the test set will receive extra credit (first place +3, second place +2, third place +1)**

**Requirements**
- Trim the sales data to those sales with prices between $3,000 and $1 million
- Set up a pipeline that includes both numerical columns and categorical columns
- Include one-hot encoded variables for the neighborhood of the sale
- Use a 70/30% training/test split
- Use GridSearchCV to perform a $k$-fold cross validation that optimize *at least 2* hyperparameters of the RandomForestRegressor
- After fitting your model and finding the optimal hyperparameters, you should evaluate the score ($R^2$) on the test set (the original 30% sample withheld)

**Notes**

- You are welcome to include additional features or perform any feature engineering that you want to try to improve the test accuracy
- You can also experiment with the prediction variable, e.g., try predicting sale price per sq ft. (or its log)

### 2.5 Calculate the percent error of your model predictions for each sale in the test set

Fit your best model and use it to make predictions on the test set.

**Note:** this should be the percent error in terms of **sale price** or **sale price per sq ft**. You'll need to convert if you predicted the log!

### 2.6 Make a data frame with percent errors and census tract info for each sale in the test set

Create a data frame that has the property geometries, census tract data, and percent errors for all of the sales in the test set.

**Notes**

- When using the "train_test_split()" function, the index of the test data frame includes the labels from the original sales data frame
- You can use this index to slice out the test data from the original sales data frame, which should include the census tract info and geometries
- Add a new column to this data frame holding the percent error data



### 2.8 Plot a map of the median percent error by census tract 

- You'll want to group your data frame of test sales by the `GEOID10` column and take the median of you percent error column
- Merge the census tract geometries back in and use geopandas to plot.

### 2.9 Compare the percent errors in Qualifying Census Tracts and other tracts 

[Qualifying Census Tracts](https://www.huduser.gov/portal/datasets/qct.html) are a poverty designation that HUD uses to allocate housing tax credits

- I've included a list of the census tract names that qualify in Philadelphia in 2019
- Add a new column to your dataframe of test set sales that is True/False depending on if the tract is a QCT
- Then, group by this new column and calculate the median percent error

**You should find that the algorithm's accuracy is significantly worse in these low-income, qualifying census tracts**

In [1]:
qct = ['5',
 '20',
 '22',
 '28.01',
 '30.01',
 '30.02',
 '31',
 '32',
 '33',
 '36',
 '37.01',
 '37.02',
 '39.01',
 '41.01',
 '41.02',
 '56',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '69',
 '70',
 '71.01',
 '71.02',
 '72',
 '73',
 '74',
 '77',
 '78',
 '80',
 '81.01',
 '81.02',
 '82',
 '83.01',
 '83.02',
 '84',
 '85',
 '86.01',
 '86.02',
 '87.01',
 '87.02',
 '88.01',
 '88.02',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '98.01',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '119',
 '121',
 '122.01',
 '122.03',
 '131',
 '132',
 '137',
 '138',
 '139',
 '140',
 '141',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '151.01',
 '151.02',
 '152',
 '153',
 '156',
 '157',
 '161',
 '162',
 '163',
 '164',
 '165',
 '167.01',
 '167.02',
 '168',
 '169.01',
 '169.02',
 '170',
 '171',
 '172.01',
 '172.02',
 '173',
 '174',
 '175',
 '176.01',
 '176.02',
 '177.01',
 '177.02',
 '178',
 '179',
 '180.02',
 '188',
 '190',
 '191',
 '192',
 '195.01',
 '195.02',
 '197',
 '198',
 '199',
 '200',
 '201.01',
 '201.02',
 '202',
 '203',
 '204',
 '205',
 '206',
 '208',
 '239',
 '240',
 '241',
 '242',
 '243',
 '244',
 '245',
 '246',
 '247',
 '249',
 '252',
 '253',
 '265',
 '267',
 '268',
 '271',
 '274.01',
 '274.02',
 '275',
 '276',
 '277',
 '278',
 '279.01',
 '279.02',
 '280',
 '281',
 '282',
 '283',
 '284',
 '285',
 '286',
 '287',
 '288',
 '289.01',
 '289.02',
 '290',
 '291',
 '293',
 '294',
 '298',
 '299',
 '300',
 '301',
 '302',
 '305.01',
 '305.02',
 '309',
 '311.01',
 '312',
 '313',
 '314.01',
 '314.02',
 '316',
 '318',
 '319',
 '321',
 '325',
 '329',
 '330',
 '337.01',
 '345.01',
 '357.01',
 '376',
 '377',
 '380',
 '381',
 '382',
 '383',
 '389',
 '390']